# Extracting the Jeopardy Clue Base from J-Archive
<b>by Matt Jackson</b>


Dedicated fans of the TV game show <em>Jeopardy!</em> have created a resource called <a href="http://j-archive.com/">J-Archive</a>, which stores the game boards for every game aired.

Thankfully, it's pretty well-formatted data.

Each individual game is stored on a distinct page where you can view the gameboard "as-is", and mouse over each clue to reveal what the correct response was (and which player, if any, rang in to give it). Each game is a page with its own "game_id" corresponding to the order in which that game was archived; for example, the finale of Season 32 is at http://www.j-archive.com/showgame.php?game_id=5384 because it was the 5384th game to be archived.

(Confusingly, if you go to that page, you see "Show #7355", because it was the 7,355th show to <em>air</em> on television. (The game_id number is usually lower than the show number, because many old episodes from early in the show's run are now lost to time and aren't archived at all.)


We start by importing the modules we need. 
* This script uses <a href="https://www.crummy.com/software/BeautifulSoup/">BeautifulSoup</a> to convert the raw HTML of each J-Archive "game" page into a DOM object that we can extract slices from (such as the text of an individual clue).
* We also use regular expressions to isolate substrings in cases where we need to drop extraneous information surrounding the data we want to keep for each clue.
* Once we've gotten the useful information for each clue, we create a data frame in Pandas to store those clues.

In [1]:
import pandas as pd 
from bs4 import BeautifulSoup
import re
import urllib.request
import time

For each clue, I want to get an ordered n-tuple of:
* <b>Show ID</b> (this is the number at the top of the page, NOT the game_id in its url)
* <b>Air date</b>
* <b>Clue round</b> (the clue is nested within a <div> of id "jeopardy_round", double_jeopardy_round", or "final_jeopardy_round"; go up the tree from the clue to get the round it's in)
* <b>Clue category</b> (to get this, you have to: Create an ordered list of duples in which each category string from the first to the thirteenth is matched up with "J_1," "J_2," ... "DJ_1" ..., "FJ". THEN truncate the id element of "clue_text" down to match the relevant key. THEN output the corresponding category name and drop that into the table.</b>
* <b>Clue value</b> (to get this you have to go back up to "clue_value" and drop that in as a string)</b>
    * <b>Is clue a DD?</b> (Put in "DD" if there is an element of class "clue_value_daily_double" in the clue; otherwise don't)
* <b>Clue text</b> (The text that Alex Trebek reads) (get a string of "clue_text")</b>
* <b>Correct Response</b> (get the portion of the <div onclick=...> tag which is between "correct_response&quot;&gt;" and "&lt;" -- this may be a bit of a pain)
    
Putting some auxiliary procedures up front to help with that:

In [2]:
def getRound(str):
    if bool(re.search('^DJ', str)) == True:
        return "Double Jeopardy"
    elif bool(re.search('^FJ', str)) == True:
        return "Final Jeopardy"
    elif bool(re.search('^J', str)) == True:
        return "Single Jeopardy"
    else:
        return "None"

In [3]:
def getValue(cluesoup):
    if cluesoup.find("td", {"class": "clue_value_daily_double"}) != None:
        return 'Daily Double'
    elif cluesoup.find("td", {"class": "clue_value"}) != None:
        return cluesoup.find("td", {"class": "clue_value"}).text
    else:
        return ""

In [1]:
def getCorrectResponse(indivClue, soup):
    boardID = indivClue.find("td", {"class": "clue_text"})['id']
    boardIndex = re.search('(J|DJ|FJ)(_[1-6])?', boardID).group()
    if getRound(boardIndex) == 'Final Jeopardy':
        #Final is coded weirdly enough compared to everything else that you need some way to divert what comes out here to something else if it's final
        textDirty = soup.find_all("div")[-2]['onmouseover']
        textGood = textDirty.split('em')[1][28:-2]
        return textGood
    else:
        textDirty = indivClue.find("div")['onmouseover']
        textGood = textDirty.split('em')[1][26:-2]
        return textGood

Here is the main procedure for turning a single "game" of Jeopardy (a page on J-Archive) into a .csv with one row per clue of the game.

In [12]:
#There are a bunch of print statements for debugging purposes. I've commented those out

def game_scrape(game_num):
    
    #Open the page we're scraping the clues from.
    game_page = urllib.request.urlopen('http://www.j-archive.com/showgame.php?game_id=' + str(game_num))
    
    #Use BeautifulSoup to create a 'soup' object of that page. By convention, name it "soup"
    soup = BeautifulSoup(game_page, "lxml")
    #print(soup.prettify())
    
    # Extract the show number and the date on which it aired from the "title" div at the top of the page.
    shownum = soup.find("title").text.split(' ')
    showID = shownum[4][1:-1:]
    airDate = shownum[6]
    # print(airDate)
    # print(showID)
    
    # Extract the list of all 13 categories used in the round
    # (Six categories in the "Single" Jeopardy round, six in Double Jeopardy, and one for Final Jeopardy.)
    category_names = [element.text for element in soup.find_all("td", {"class": "category_name"})]
    # print(category_names)
    # Store those categories in a dictionary paired with their corresponding column on the 6-column board
    cat_values = ['J_1', 'J_2', 'J_3', 'J_4', 'J_5', 'J_6', 'DJ_1', 'DJ_2', 'DJ_3', 'DJ_4', 'DJ_5', 'DJ_6', 'FJ']
    cat_dict = dict(zip(cat_values, category_names))
    # print(cat_dict)
    
    # Get the raw HTML block for each of the 61 squares into an array, to be worked with later
    cluesFULL = [element for element in soup.find_all("td", {"class": "clue"})]
    # print(len(cluesFULL))
    
    """Sometimes the timer will go off before all clues have been read.
    Further steps later on in the script will fail if blank clues are included.
    Plus, we don't want them anyway, because there's no use in studying a clue 
    with no text. So we're going to cull blank clues out in advance."""
    
    blank_clue = '<td class="clue">\n</td>'
    #loop within the loop - this one loops all clues within the current game
    for i in range(len(cluesFULL) -1):
        if str(cluesFULL[i]) == blank_clue:
        #turns each clue that was not called up into an empty string
            cluesFULL[i] = ''
    #using None a a filter element will remove all empty strings, leaving behind only the clues that were heard in a game
    cluesFULL = list(filter(None, cluesFULL))
    # print(cluesFULL)
    # print(len(cluesFULL))
    
    """Create a list of lists to store all clue information from this one game.
    It's blank at first."""
    
    game_cards_FULL = []
    
    #There may be a way to append direct to a data frame using pandas methods, which would be more efficient
    for i in range(len(cluesFULL)):
    #would it save memory to have getBoardIndex be a procedure like the others?
        boardID = cluesFULL[i].find("td", {"class": "clue_text"})['id']
        boardIndex = re.search('(J|DJ|FJ)(_[1-6])?', boardID).group()
        round = getRound(boardIndex)
        category = cat_dict.get(boardIndex)
        value = getValue(cluesFULL[i])
        clue_text = cluesFULL[i].find("td", {"class": "clue_text"}).text
        correct_response = getCorrectResponse(cluesFULL[i], soup)
        singleCardArray = [showID, airDate, category, round, value, clue_text, correct_response]
        game_cards_FULL.append(singleCardArray)
        
     # print(game_cards_FULL)
    
    # Convert the array of clue n-tuples into a pandas data frame.
    df_this_game = pd.DataFrame(game_cards_FULL, columns=['GameID', 'Air Date', 'Category', 'Round', 'Value', 'Clue Text', 'Correct Response'])

    # return the data frame as the output of this function
    return df_this_game
    

In [6]:
#initialize an empty data frame to hold all the cards from all the games
df_whole_series = pd.DataFrame(columns=['GameID', 'Air Date', 'Category', 'Round', 'Value', 'Clue Text', 'Correct Response'])

In [7]:
# game_scrape(1)


In [8]:
#Create a for loop to scrape each game, append it to a giant data frame of all the clues across games 

for game_num in range(1,11):
    
    #J-archive's robots.txt has a setting of crawl-delay: 20; is this script technically a robot? 
    #If so, I believe I have to include this to respect that:
    
    # time.sleep(20)
    
    print("About to scrape game " + str(game_num))
    
    df_to_add = game_scrape(game_num)
    
    df_whole_series = df_whole_series.append(df_to_add)
    print("Game " + str(game_num) + " scraped!")
    


About to scrape game 1
Game 1scraped!
About to scrape game 2
Game 2scraped!
About to scrape game 3
Game 3scraped!
About to scrape game 4
Game 4scraped!
About to scrape game 5
Game 5scraped!
About to scrape game 6
Game 6scraped!
About to scrape game 7
Game 7scraped!
About to scrape game 8
Game 8scraped!
About to scrape game 9
Game 9scraped!
About to scrape game 10
Game 10scraped!


In [9]:
#Preview the data frame of clues 

# df_whole_series

,GameID,Air Date,Category,Round,Value,Clue Text,Correct Response
0,4596,2004-09-06,THE OLD TESTAMENT,Single Jeopardy,$200,"Let's all flock to read Psalm 95, in which hum...",sheep
1,4596,2004-09-06,SEAN SONG,Single Jeopardy,$200,"The rap on him is he's sometimes ""Puffy""",Sean Combs
2,4596,2004-09-06,CITY WALK,Single Jeopardy,$200,In the 1980s this city passed Chicago as the U...,Los Angeles
3,4596,2004-09-06,SLOGANEERING,Single Jeopardy,$200,"""Ask any mermaid you happen to see, 'What's th...",Chicken of the Sea
4,4596,2004-09-06,ROOM,Single Jeopardy,$200,"Many people have become familiar with ""Shape"",...",magazines
5,4596,2004-09-06,"""BOARD""",Single Jeopardy,$200,"In 2003, Roy E. Disney left it after being on ...",the Disney Board of Directors
6,4596,2004-09-06,THE OLD TESTAMENT,Single Jeopardy,$400,Some say the screech owl in the King James Bib...,Adam
7,4596,2004-09-06,SEAN SONG,Single Jeopardy,$400,Once married to Madonna,Sean Penn
8,4596,2004-09-06,CITY WALK,Single Jeopardy,$400,"This London borough is the G in GMT, squire",Greenwich
9,4596,2004-09-06,SLOGANEERING,Single Jeopardy,$400,"A fast food chain: ""If it doesn't get all over...",Carl\'s Jr.


In [10]:
#Write the data frame of every clue ever to a csv on your hard drive.

# df_whole_series.to_csv('J-Archive clue base.csv', index=False)

## Then what do you do with it?

Take the giant csv you just made, and import it into <a href="https://apps.ankiweb.net/docs/manual.html#adding-cards-and-notes">Anki</a> using its bulk import feature. Name the deck "Jeopardy" or similar.

To quiz myself with these, I created a custom card format in Anki with "Correct Response" on the back of the card and all other fields on the front.


## Errors


* The script timed out after game 803, 2190
* Games 1152, 1473, 2172, 3081, 3889, 4590, 5416, and 5922 produce this error:

```
AttributeError Traceback (most recent call last)
<ipython-input-10-039705e691b0> in <module>()
----> 1 extractGames(804,6112,'J-Archive clue base ENTIRE.csv')

<ipython-input-8-a0233dc08c10> in extractGames(start, finish, dest_file_string)
     92         #would it save memory to have getBoardIndex be a procedure like the others?
     93             boardID = cluesFULL[i].find("td", {"class": "clue_text"})['id']
---> 94             boardIndex = re.search('(J|DJ|FJ)(_[1-6])?', boardID).group()
     95             round = getRound(boardIndex)
     96             category = cat_dict.get(boardIndex)

AttributeError: 'NoneType' object has no attribute ‘group’
```


* Games 2195, 3063, 3081, 3552, 4246, 4256, 4271, 5348, 5885, 6054, 6061, 6064, 6065, 6067, and 6085 produce this error:

```
TypeError                                 Traceback (most recent call last)
<ipython-input-4-f06816e9bd73> in <module>()
----> 1 extractGames(2190,6112,'J-Archive clue base ENTIRE.csv')

<ipython-input-1-a0233dc08c10> in extractGames(start, finish, dest_file_string)
     91         for i in range(len(cluesFULL)):
     92         #would it save memory to have getBoardIndex be a procedure like the others?
---> 93             boardID = cluesFULL[i].find("td", {"class": "clue_text"})['id']
     94             boardIndex = re.search('(J|DJ|FJ)(_[1-6])?', boardID).group()
     95             round = getRound(boardIndex)



TypeError: 'NoneType' object is not subscriptable
```

* For game #3576, only “U.S. Cities” loaded as a category in Double Jeopardy — other five are blank]
* For some early games (like games that aired in the 1980s), the clue and response value are just the “=” sign with no other content

I haven't quite gotten to the bottom of exactly what causes these errors -- there's probably something a little different in the HTML of the pages for those games, like an unexpected tag which runs into my code excepting text from tags and messes it up. I wanted to get my flash cards set up as soon as I could, so I went ahead and restarted the "for" loop with a new starting value to skip over each of the games affected by the big errors. This means my database of clues is missing about 23 games out of 6100+.
